# Entrega 3 - Clasificador con Aprendizaje Bayesiano y KNN

### Grupo 6:
     - Renzo Beux 5076905-8
     - Darío Cruz  4768599-2
     - Agustín Tornaría 5047825-5

## 1. Objetivo

El objetivo de esta tarea es construir un clasificador utilizando los algoritmos de Naives Bayes y KNN. 

Para medir el éxito del aprendizaje se separa el conjunto de datos en 2 subconjuntos, uno de entrenamiento el cuál se utiliza para entrenar y otro de testing que solo se utiliza para medir el éxito, es importante destacar que estas mediciones no se utilizan para entrenar. Estos subconjuntos fueron separados manteniendo una proporción de 80% de los datos para entrenamiento y 20% para testing.

## 2. Diseño

## 2.1 Preprocesamiento de datos
<!-- - Decisiones sobre tratamiento de datos numéricos, faltantes, etc. antes de la aplicación de el algoritmo
- Selección/generación de atributos -->

Los datos son presentados en una planilla CSV que se consume utilizando Pandas y luego se aplican transformaciones a los datos.

Primero se observan los valores que toman los atributos y se clasifican según si son categóricos, si importa el orden y si contienen datos unknown. A su vez se agrega un análisis de a que distribución se aproximan más los datos (bloque de código que se encuentra debajo de este bloque).

<table>
  <tr style="font-weifht:bold">
    <th>Atributo</th>
    <th>Categórico</th>
    <th>Importa el Orden</th>
    <th>Contiene Unknown</th>
    <th>Distribución</th>
  </tr>
  <tr>
    <td>age_range</td>
    <td>si</td>
    <td>si</td>
    <td>no</td>
    <td>Gaussiana</td>
  </tr>
  <tr>
    <td>balance_range</td>
    <td>si</td>
    <td>si</td>
    <td>no</td>
    <td>-</td>
  </tr>
  <tr>
    <td>job</td>
    <td>si</td>
    <td>no</td>
    <td>si</td>
    <td>-</td>
  </tr>
  <tr>
    <td>siblings</td>
    <td>si</td>
    <td>no</td>
    <td>no</td>
    <td>Gaussiana</td>
  </tr>
  <tr>
    <td>marital</td>
    <td>si</td>
    <td>no</td>
    <td>no</td>
    <td>Gaussiana</td>
  </tr>
  <tr>
    <td>education</td>
    <td>si</td>
    <td>si</td>
    <td>si</td>
    <td>Gaussiana</td>
  </tr>
  <tr>
    <td>default</td>
    <td>si</td>
    <td>no</td>
    <td>no</td>
    <td>Gaussiana</td>
  </tr>
  <tr>
    <td>housing</td>
    <td>si</td>
    <td>no</td>
    <td>no</td>
    <td>Gaussiana</td>
  </tr>
  <tr>
    <td>loan</td>
    <td>si</td>
    <td>no</td>
    <td>no</td>
    <td>Gaussiana</td>
  </tr>
  <tr>
    <td>contact</td>
    <td>si</td>
    <td>no</td>
    <td>si</td>
    <td>Gaussiana</td>
  </tr>
  <tr>
    <td>month</td>
    <td>si</td>
    <td>si*</td>
    <td>no</td>
    <td>-</td>
  </tr>   
  <tr>
    <td>campaign</td>
    <td>no</td>
    <td>si</td>
    <td>no</td>
    <td>Pareto</td>
  </tr> 
  <tr>
    <td>pdays</td>
    <td>no</td>
    <td>si</td>
    <td>no</td>
    <td>-</td>
  </tr>
  <tr>
    <td>previous</td>
    <td>no</td>
    <td>si</td>
    <td>no</td>
    <td>-</td>
  </tr>
  <tr>
    <td>poutcome</td>
    <td>si</td>
    <td>no</td>
    <td>si</td>
    <td>Pareto</td>
  </tr>
  <tr>
    <td>duration</td>
    <td>si</td>
    <td>si</td>
    <td>no</td>
    <td>-</td>
  </tr>
      <caption>Tabla 1 - Clasificación de atributos</caption>
</table>

Se observa que todos los atributos son categóricos a excepción de 3: campaign, pdays y previous; además estos 3 atributos no contienen unknown y toman valores enteros, por lo tanto se decide dejarlos como están.
El resto de los atributos son categóricos y son de tipo "String" a excepción de Siblings, si importa el orden se transforman a enteros asignando un número distinto a cada posible valor que toma el atributo, empezando en 0 y luego 1, 2, ... etc; respetando el orden (por ejemplo para balance_range se transformó 'low' en 0, 'medium' en 1 y 'high' en 2). Para el caso del atributo 'month' el orden que se quiere establecer es circular, entonces se transforma 'jan' en 1, 'feb' en 2, ... etc y al calcular la distancia se tiene en cuenta lo mencionado.

A los atributos que no importa el orden se podría haber aplicado one hot encoding, pero se decidió no hacerlo, en cambio al momento de aplicar KNN se define una distancia teniendo en cuenta la naturaleza de cada atributo. Si utilizamos la norma euclidiana, pero en lugar de calcular simplemente la diferencia entre coordenadas, realizamos lo siguiente:

- Para atributos categóricos, donde no importa el orden, retornamos 0 si es el mismo valor o 1 si son valores diferentes.
- En el caso particular de la fecha, consideramos que es un atributo categórico y está ordenado, pero además es cíclico. Por ejemplo, la distancia entre Noviembre y Enero es 2, no 10.
- Para los otros casos simplemente realizamos la resta entre los dos valores

Además entre los atributos categóricos hay 4 que toman el valor unkown y hay distintos criterios que se podrían haber aplicado para manejar este valor como podría ser eliminar los datos que tienen unknow, tomarlo como otro valor posible, sustituirlo por el valor más ocurrente o buscar el dato que más se parezca y asignarle el mismo valor en ese atributo. Para los 4 casos que se tienen se tomó 'unknown' como otro valor posible, en el caso de los atributos 'job', 'education' y 'telephone' se toma esta decisión dado que se interpretó que se desconoce o que el individuo no proporcionó esa información, en el caso de 'poutcome' no se está seguro del significado pero como representa el 84.6% de los valores también se trata de la misma manera, no sería una buena idea quitarlos ya que se perderían la mayoría de datos, tampoco se cree adecuado tomar el valor más común ya que hay muy pocos datos de referencia y elegirlo según el resto de datos/atributos sería muy poco performante.


Con el motivo de evitar sobreajustes es por lo que se va a dividir el conjunto de datos en 2, Testeo y Entrenamiento. Esto se lleva a cabo para que el modelo no resulte preciso con los datos los cuales se entrenó y errado para cualquier otro dato nuevo que se le presente. Lo más común es una división en 80% de entrenamiento y 20% de testeo, que es la que se usó.

A su vez se separa el conjunto de entrenamiento en dos, el conjunto de entrenamiento propiamente dicho y el conjunto de validación. La idea principal del conjunto de validación es el poder ajustar los hiperparámetros utilizando este conjunto para evaluar lo entrenado con el conjunto de entrenamiento sin estar completamente sesgado debido al overfitting, cosa que sucedería si se usara el mismo conjunto con el que se entrenó. 
Como se pretende utilizar validación cruzada, lo que se hace es dividir el conjunto de entrenamiento en N partes y luego entrenar con n-1 partes y la enésima parte es tomada como validación y lo que se realiza es ir rotando el conjunto de validación.

Es importante el paso de estratificación ya que la mayoría de los valores objetivos toman el valor “no”. Para ello se impone en el código que las proporciones de "yes-no" en las divisiones del conjunto de datos se mantengan.




In [ ]:
#Atención el siguiente modulo utiliza scipy
!python parte1.py

## 2.2 Algoritmo
<!-- Extensiones del algoritmo original necesarias para la resolución del problema: tratamiento de atributos faltantes, numéricos, etc. (si es el propio algoritmo el que lo maneja), implementaciones adicionales necesarias para manejar ensambles de clasificadores, etc. -->

Los dos algoritmos que se prueban son el de Naives Bayes y KNN. Para ambos se implementa una versión propia y otra utilizando SciKit.

En el algoritmo propio de Naives Bayes primero se verificó que las columnas fueran independientes y lo eran, luego se experimentó sacar distintos atributos para ver si mejoraba, en particular algunos casos donde se creía que dos atributos tenían una correlación o un atributo que tomaba el mismo valor en la mayoría de los datos, pero en todos los casos empeoró la medidaF y por lo tanto no se sacaron columnas. 

Para el algoritmo propio de KNN se utiliza una definición de distancia basada en la euclideana pero con algunas modificaciones. 
Para el atributo 'month' se tiene en cuenta la circularidad tomando en cuenta que luego del '12' viene el '1', así se obtiene por ejemplo que la distancia entre '11' y '2' es 3. 
Para los atributos categóricos que no importa el orden se mencionó anteriormente que no se utilizó one hot encoding, lo que se hizo es usar la distancia discreta, es decir 0 si son iguales, 1 si son distintos.
Y para el resto de los atributos se utiliza el valor absoluto de la resta.
De esta manera la distancia entre dos puntos cualquiera se calcula como la suma de los cuadrados de la distancia entre cada atributo.

Además se utiliza validación cruzada para intentar elegir el mejor k entre {1,3,5,7,9}, para ello se divide el conjunto de entrenamiento en 4 partes iguales, se toma uno de los conjuntos para validar y los otros 3 para clasificar los puntos del conjunto de validación con los distintos k's, se mide la performance para cada k. Esto se repite cambiando el conjunto de validación entre los 4 conjuntos separados anteriormente y se hace el promedio, con lo que se elige el k.


## 2.3 Evaluación
<!-- - Qué conjunto de métricas se utilizan para la evaluación de la solución y su definición
- Sobre qué conjunto(s) se realiza el entrenamiento, ajuste de la solución, evaluación, etc. Explicar cómo se construyen estos conjuntos. -->

Las métricas utilizadas son exactitud, precisión, recall y medida-F. Para ello se divide el conjunto en dos, uno de training (80%) y otro de testing (20%) utilizando estratificación, asegurando que haya el mismo porcentaje de valores "si" y "no" en cada uno de los conjuntos. La medida 'medida F' es la métrica utilizada para evaluar en el cross-validation los hiperparámetros, se elige el mejor promedio de esta métrica para seleccionar el hiperparámetro para luego predecir utilizando el conjunto de testeo.
Una vez generada la predicción se evalúan en término de las métricas anteriormente dichas y se imprimen en consola.



## 3. Experimentación

Al momento de entrenar los algoritmos de Bayes, tanto con nuestra implementación y como la implementación de scikit, se realizó validación cruzada partiendo el conjunto de entrenamiento en 10 partes. Realizamos distintas configuraciones de los hiper parámetros: cantidad de columnas y la estimación a priori de la probabilidad buscada.

A continuación listamos las diferentes configuraciones:
1. Experimento 0: Sin quitar columnas y todos los valores equiprobables
2. Experimento 1: Probando quitar atributos aparentemente correlacionados
3. Experimento 2: Probando quitar atributos con poca dispersión
4. Experimento 3: Probando usar distribuciones diferentes a la equiprobable

En el experimento 1 probamos quitar o bien el atributo 'pdays' o el atributo 'poutcome', ya que en el análisis arrojaron que estaban correlacionados en un proporción mayor al 80%

En el experimento 2, los atributos que probamos remover son los 'default', 'loan', 'campaign', 'pdays', 'previous' y 'poutcome'.

En el experimento 3, para las columnas que al analizarlas nos dieron una distribución normal gaussiana, utilizamos una estimación de la probabilidad del tipo gaussiana, en lugar de usar la equiprobable.

Luego de realizar todos los experimentos, seleccionamos el modelo con los hiperarámetros que tuvo una mayor medidaF. Pero si ese modelo no tuvo una mediaF notoriamente mayor que el el modelo sin modificaciones (osea el experimento 0) entonces el modelo sin modificaciones es con el que nos quedamos. Para nosotros una mediaF notoriamente mayor significa que supera al menos en un 1%.

Para los algoritmos de KNN tantos nuestros como los de SciKit se hizo validación cruzada separando el conjunto de entrenamiento en 4 para encontrar el mejor K. En ambas implementaciones se resuelve usar el k = 1. Suponemos que el algoritmo funciona mejor con K=1 porque al haber un desbalance en los valores objetivos, es muy fácil encontrarse con valores incorrectos en el espacio. Este problema puede ser resuelto de varias formas, como puede ser SMOTE, que es una técnica de oversampling, osea crear puntos nuevos en el espacio cerca de los ya existentes para mejorar el funcionamiento del algoritmo. Las gráficas de los K en función de la medida F son las siguientes:

![knn implementado](knn_parte4_o.png)

_Valor de FMeasure en función de K para nuestra implementación_

![knn scikit](knn_parte5_o.png)

_Valor de FMeasure en función de K para SciKit_

Distintas pruebas fueron ejecutadas, tales como quitar columnas, reescalar los datos, entre otras pero no se lograron mejoras, incluso pudiendo tener empeoramiento en el comportamiento.
Donde vimos una mejora fue en el tiempo de ejecución del algoritmo de sklearn, esto se debe a que utilizan KDTrees el cual es una estructura que lleva un tiempo considerable en crearla pero luego para buscar los puntos más cercanos a un punto fijo corre en tiempo logarítmico. Esto lleva a que nuestra implementación tarde más de 1 hora en devolvernos un resultado mientras que SciKit lo realiza en cuestión de minutos. Una posible mejora a nuestro algoritmo sería el uso de distintos thread para paralelizar los cálculos de distancias y así mejorar el tiempo de respuesta.

En cualquiera de los casos la medida F es alrededor de un 30% menos en nuestro KNN que el rendimiento variaba entre 15% y 22% dependiendo de la vez que se corriera. Esto como bien expresamos anteriormente creemos que es por la naturaleza de los datos. Con un mejor preprocesamiento de los datos se conseguirían mejores resultados.

A continuación se muestra una tabla comparativa con los resultados de la experimentación, que se comentaron anteriormente.

| Algoritmo                         | accuracy | precision | recall | medidaF |
|-----------------------------------|----------|-----------|--------|---------|
| Naives Bayes - parte 3            | 90.5     | 47.97     | 26.89  | 34.47   |
| Naive Bayes - scikit (GaussianNB) | 86.73    | 30.14     | 32.58  | 31.31   |
| KNN - parte 4                     | 84.98    | 18.21     | 17.68  | 17.94   |
| KNN - scikit                      | 87.36    | 30.68     | 28.66  | 29.63   |

Como se puede observar en el caso de Bayes nuestro algoritmo tuvo un mejor desempeño, sin embargo para KNN fue notoriamente peor. Esto podría deberse a que Bayes no sufre de la "maldición de las dimensiones" como KNN, donde la presencia de atributos irrelevantes puede afectar a la precisión de KNN. Pero como se explicó anteriormente, realizamos un proceso de selección de atributos sin obtener mejoras.

Agregamos a continuación un enlace a los scripts de cada parte del laboratorio:

In [1]:
!python parte3.py

  File "parte3.py", line 117
    print(f'predición: {predict}, {extra_tab} \tvalor real: {dictio["term_deposit"]}, \tP(instancia/valor): {prob},   \tconfianza de P: {seguiridad}\n')
                                                                                                                                                       ^
SyntaxError: invalid syntax


In [ ]:
#ADVERTENCIA: Alto uso de recursos, demora mucho tiempo
!python parte4.py

In [ ]:
!python parte5.py

## 4. Conclusión

Destacamos como algo muy importante el uso de otras métricas además de la exactitud, ya que en un principio pensamos que nuestros algoritmos funcionaban muy bien , viendo que la exactitud rondaba el 90%, sin embargo al utilizar la precisión y el recall nos dimos cuenta que esta exactitud era 'mentirosa' debido a que si se implementase un algoritmo que solo responde "NO" le acertaría alrededor de 90% de las veces, ya que el porcentaje de NOs en la columna objetivo es alrededor de 90%, esto es una claro sintoma de un conjunto desbalanceado. Fue entonces cuando el uso de las otras métricas nos hicieron dar cuenta los problemas que tenian estos clasificadores.

En cuanto a Bayes, luego de probar con las distintas implementaciones de SciKit vimos que la que mejor predecía era la Gaussiana.
En cuanto a nuestra implementación de Bayes, se puede decir que al quitar atributos o probar distintas distribuciones no contribuyeron en gran medida a mejorar el algoritmo. Incluso aveces la mediaF daba peor.

Por ultimo, el uso de varios metodos de preprocesamiento de los datos es de vital importancia, en especial para casos donde los datos están desbalanceados.

<!-- Una breve conclusión del trabajo realizado. Por ejemplo: 
- ¿cuándo se dieron los mejores resultados del jugador?
- ¿encuentra alguna relación con los parámetros / oponentes/ atributos elegidos?
- ¿cómo mejoraría los resultados? -->